In [ ]:
from domains import load_domains

In [2]:
domains, eval_domains = load_domains()

In [ ]:
from lwp import LWP

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

models = []

source_dataset = domains[0]
model = LWP(distance_metric='euclidean')
model.fit(source_dataset['features'], source_dataset['labels'])
models.append(model)

for i in range(1, 20):
    features = domains[i]['features']
    
    # Clustering unlabeled data into 10 clusters
    kmeans = KMeans(n_clusters=10, random_state=42)
    clusters = kmeans.fit_predict(features)
    

    # Assign labels to clusters
    cluster_centers = kmeans.cluster_centers_
    predicted_labels = model.predict(cluster_centers)

    # Label unlabeled data
    pseudo_labels = np.array([predicted_labels[cluster] for cluster in clusters])

    model.fit(features, pseudo_labels)
    models.append(model)

In [ ]:
from sklearn.metrics import accuracy_score
import pandas as pd

df = pd.DataFrame()

for idx,model in enumerate(models) :
    
    scores = []
    for eval_domain in eval_domains[:idx+1]:
        
        features = eval_domain['features']
        labels = eval_domain['labels']
        
        preds = model.predict(features)
        acc = accuracy_score(labels, preds)
        
        scores.append(acc)
    
    df[f'Domain {idx+1}'] = scores + [np.nan] * (len(eval_domains) - len(scores))

In [ ]:
print(df)

    Domain 1  Domain 2  Domain 3  Domain 4  Domain 5  Domain 6  Domain 7  \
0      0.856    0.8560    0.8560    0.8560    0.8560    0.8560    0.8560   
1        NaN    0.8544    0.8544    0.8544    0.8544    0.8544    0.8544   
2        NaN       NaN    0.8620    0.8620    0.8620    0.8620    0.8620   
3        NaN       NaN       NaN    0.8816    0.8816    0.8816    0.8816   
4        NaN       NaN       NaN       NaN    0.8744    0.8744    0.8744   
5        NaN       NaN       NaN       NaN       NaN    0.8828    0.8828   
6        NaN       NaN       NaN       NaN       NaN       NaN    0.8664   
7        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
8        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
10       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
11       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
12       NaN